In [1]:
import pandas as pd
import numpy as np
import gdal
from gdalconst import * 
import os, sys, time
import xlsxwriter

C:\Anaconda\lib\site-packages\pandas\computation\expressions.py:21: UserWarning: The installed version of numexpr 2.0.1 is not supported in pandas and will be not be used
The minimum supported version is 2.1

  "version is 2.1\n".format(ver=ver), UserWarning)


In [2]:
def getValues(rasterDS, locData, rasterName):
    size = len(locData)
    startTime = time.time() #if timing the function speed is really necessary
    xValues = locData.loc[:, 'Xmax']
    yValues = locData.loc[:, 'Ymax']
    rows = rasterDS.RasterYSize #number of rows
    cols = rasterDS.RasterXSize # number of columns
    bands = rasterDS.RasterCount # number of data value types (it's going to be 1 for our case)
    
    # get georeference info
    transform = rasterDS.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = transform[5]
    
    band = rasterDS.GetRasterBand(1)
    # getting all data at once
    allData = band.ReadAsArray(0,0, cols, rows)
    
    #Result dataset
    reqDataset = locData
    reqDataset[rasterName] = 'nan'
    
    for i in range(size):
        # get x, y
        x = xValues[i]
        y = yValues[i]
        # compute pixel offset
        xOffset = int((x - xOrigin) / pixelWidth)
        yOffset = int((y - yOrigin) / pixelHeight)
        value   = allData[yOffset, xOffset]
        reqDataset.loc[i, rasterName] = value
    endTime = time.time()
    print 'The script took ' + str(endTime - startTime) + 'seconds'
    
    return reqDataset

In [3]:
# baseFile = 'C:\\Users\\Allan\\Dropbox\\2015\\Research Data\\Weather Variables\\'
# coordDataset = dataset
# link = '\\RasterFiles\\'
def getDataset(RasterID, cat, coordDataset, baseFile, link):
    rasterName = str(RasterID) # RasterID might be either a string(month) or an int (year)
    fileName = baseFile + cat + link + rasterName + '.asc'
    ds = gdal.Open(fileName)
    if ds is None:
        print 'could not open ' + fileName
        sys.exit(1)
    completeDataset = getValues(ds, coordDataset, rasterName)
#     return completeDataset

In [4]:
def RasterFilesData2Excel(variable, coordDataFrame):
    baseFile = 'C:\\Users\\Allan\\Dropbox\\2015\\Research Data\\RasterFiles\\'
    resFolder = 'C:\\Users\\Allan\\Dropbox\\2015\\Research Data\\Weather Variables\\'
    cal      = {1:'jan', 2: 'feb', 3:'mar', 4:'apr', 5:'may', 6:'jun', 7:'jul', 8:'aug', 9: 'sep',
               10:'oct', 11:'nov', 12:'dec'}
    if variable == 'Precipitation':
        link = '\\'
        for year in np.arange(33)+1981:
            getDataset(year, variable, coordDataFrame, baseFile, link)
        fileName = resFolder+ 'Ppt\\' + 'PrecipitationRecord1981_2013.xlsx'
        writer = pd.ExcelWriter(fileName)
        coordDataFrame.to_excel(writer)
    elif variable == 'MinTemp':
        fileName = resFolder + variable + '\\'
        for year in np.arange(33)+1981:
            link = '\\' + str(year)+'\\'
            record = fileName + str(year) + '.xlsx'
            for month in np.arange(12)+1:
                getDataset(cal[month], variable, coordDataFrame, baseFile, link)
            writer = pd.ExcelWriter(record)
            coordDataFrame.to_excel(writer, sheet_name=str(year))
            coordDataFrame = dataset[['Xmin', 'Xmax', 'Ymin', 'Ymax']]
    elif variable == 'MaxTemp':
        fileName = resFolder + variable + '\\'
        for year in np.arange(33)+1981:
            link = '\\' + str(year)+'\\' 
            record = fileName + str(year) + '.xlsx'
            for month in np.arange(12)+1:
                getDataset(cal[month], variable, coordDataFrame, baseFile, link)
            writer = pd.ExcelWriter(record)
            coordDataFrame.to_excel(writer, sheet_name=str(year))
            coordDataFrame = dataset[['Xmin', 'Xmax', 'Ymin', 'Ymax']]
    else:
        print 'Incorrent variable name. The accepted variable names include "Precipitation", "MinTemp", and "MaxTemp"' 
        
    

In [5]:
dataFile = 'C:\\Users\\Allan\\Dropbox\\2015\\Research Data\\RasterFiles\\Precipitation\\' + 'Datasets\\ppt.xlsx'
dataset = pd.read_excel(dataFile, sheetname = '1981')

In [6]:
PrecipitationDS = dataset[['Xmin', 'Xmax', 'Ymin', 'Ymax']]
MinTemperatureDS = dataset[['Xmin', 'Xmax', 'Ymin', 'Ymax']]
MaxTemperatureDS = dataset[['Xmin', 'Xmax', 'Ymin', 'Ymax']]

In [7]:
RasterFilesData2Excel('MinTemp', MinTemperatureDS)

The script took 6.71700000763seconds
The script took 6.50899982452seconds
The script took 6.45700001717seconds
The script took 6.51099991798seconds
The script took 6.5150001049seconds
The script took 6.39700007439seconds
The script took 6.39599990845seconds
The script took 6.50999999046seconds
The script took 6.41200017929seconds
The script took 6.48400020599seconds
The script took 6.39999985695seconds
The script took 6.49900007248seconds
The script took 6.58100008965seconds
The script took 6.46899986267seconds
The script took 6.44200015068seconds
The script took 6.4929997921seconds
The script took 6.4880001545seconds
The script took 6.42899990082seconds
The script took 6.43000006676seconds
The script took 6.57699990273seconds
The script took 6.59200000763seconds
The script took 6.57500004768seconds
The script took 6.50699996948seconds
The script took 6.61299991608seconds
The script took 6.8029999733seconds
The script took 6.52800011635seconds
The script took 6.58899998665seconds
The s

In [8]:
RasterFilesData2Excel('MaxTemp', MaxTemperatureDS)

The script took 6.757999897seconds
The script took 6.57299995422seconds
The script took 6.45499992371seconds
The script took 6.55700016022seconds
The script took 6.51799988747seconds
The script took 6.44499993324seconds
The script took 6.44099998474seconds
The script took 6.55700016022seconds
The script took 6.44000005722seconds
The script took 6.47499990463seconds
The script took 6.45799994469seconds
The script took 6.52300000191seconds
The script took 6.67199993134seconds
The script took 6.52100014687seconds
The script took 6.52199983597seconds
The script took 6.57299995422seconds
The script took 6.52399992943seconds
The script took 6.46399998665seconds
The script took 6.69200015068seconds
The script took 6.8820002079seconds
The script took 6.60899996758seconds
The script took 6.42999982834seconds
The script took 6.51900005341seconds
The script took 6.58200001717seconds
The script took 6.81700015068seconds
The script took 6.55500006676seconds
The script took 6.55499982834seconds
The 

In [9]:
RasterFilesData2Excel('Precipitation', PrecipitationDS)

The script took 6.58599996567seconds
The script took 6.50199985504seconds
The script took 6.53100013733seconds
The script took 6.52699995041seconds
The script took 6.49900007248seconds
The script took 6.52200007439seconds
The script took 6.48699998856seconds
The script took 6.50600004196seconds
The script took 6.48300004005seconds
The script took 6.48900008202seconds
The script took 6.48600006104seconds
The script took 6.59300017357seconds
The script took 6.51300001144seconds
The script took 6.55099987984seconds
The script took 6.47400021553seconds
The script took 6.51499986649seconds
The script took 6.49000000954seconds
The script took 6.52699995041seconds
The script took 6.46499991417seconds
The script took 6.50800013542seconds
The script took 6.46600008011seconds
The script took 6.49099993706seconds
The script took 6.492000103seconds
The script took 6.52700018883seconds
The script took 6.46000003815seconds
The script took 6.47900009155seconds
The script took 6.45199990273seconds
The